In [1]:
import mlflow
import os
import pandas as pd
from mlflow.deployments import get_deploy_client
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from mlflow_watsonml.deploy import WatsonMLDeploymentClient

# WML Deployment using MLflow

### Step 1 - get watsonml plugin from mlflow

In [2]:
client: WatsonMLDeploymentClient = get_deploy_client("watsonml")

### Step 2 - train a machine learning model and save it using mlflow
here we train a simple sklearn model

In [3]:
iris = load_iris()
X = iris.data[:, :2]  # we only take the first two features.
y = iris.target
linear_lr = LogisticRegression()
linear_lr.fit(X, y)

LogisticRegression()

In [4]:
model_path = os.path.abspath("./linear_lr")
if not os.path.exists(model_path):
    mlflow.sklearn.save_model(linear_lr, model_path)

### Step 3 - create a new WML deployment

In [9]:
deployment_name = "mlflow_demo"

In [5]:
deployment_details = client.create_deployment(name=deployment_name, model_uri=model_path)



#######################################################################################

Synchronous deployment creation for uid: 'c8157d04-d19a-4f63-8b69-a6da3af20644' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4530f21e-6276-4c9a-9fa6-563f6a9e48e8'
------------------------------------------------------------------------------------------------




In [6]:
deployment_details

{'entity': {'asset': {'id': 'c8157d04-d19a-4f63-8b69-a6da3af20644'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b',
   'name': 'S',
   'num_nodes': 1},
  'name': 'mlflow_demo',
  'online': {},
  'space_id': '0791bdd6-a2ef-4c9c-bef5-0397ecb72da3',
  'status': {'online_url': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/4530f21e-6276-4c9a-9fa6-563f6a9e48e8/predictions'},
   'serving_urls': ['https://us-south.ml.cloud.ibm.com/ml/v4/deployments/4530f21e-6276-4c9a-9fa6-563f6a9e48e8/predictions'],
   'state': 'ready'}},
 'metadata': {'created_at': '2022-12-05T21:05:58.003Z',
  'id': '4530f21e-6276-4c9a-9fa6-563f6a9e48e8',
  'modified_at': '2022-12-05T21:05:58.003Z',
  'name': 'mlflow_demo',
  'owner': 'IBMid-50R06JE29J',
  'space_id': '0791bdd6-a2ef-4c9c-bef5-0397ecb72da3'},
 'system': {'warnings': [{'id': 'Deprecated',
    'message': 'online_url is deprecated and will be removed in a future release. Use serv

### Step 4 - get predictions from the deployed model

In [10]:
predictions = client.predict(deployment_name=deployment_name, df=pd.DataFrame(X))

In [12]:
from sklearn.metrics import classification_report

print(classification_report(y, predictions["prediction"].values))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.73      0.74      0.73        50
           2       0.73      0.72      0.73        50

    accuracy                           0.82       150
   macro avg       0.82      0.82      0.82       150
weighted avg       0.82      0.82      0.82       150

